In [ ]:
%%capture
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq transformers onnx onnxruntime
    !pip install git+https://github.com/huggingface/optimum.git#egg=optimum[onnxruntime]

In [ ]:
from huggingface_hub import notebook_login, hf_hub_url

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
model_id = "arampacha/gpt-neo-therapist-small"

In [ ]:
%%capture
if 'google.colab' in sys.modules:
    !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
    !apt-get install git-lfs -y

In [ ]:
!git lfs install

Git LFS initialized.


In [ ]:
from huggingface_hub import Repository

local_dir = model_id.split("/")[-1]
repo = Repository(local_dir=local_dir, clone_from=model_id)

In [ ]:
!ls $local_dir

config.json  model-quantized.onnx  runs			    tokenizer.json
merges.txt   pytorch_model.bin	   special_tokens_map.json  training_args.bin
model.onnx   README.md		   tokenizer_config.json    vocab.json


In [ ]:
!pip install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     |████████████████████████████████| 169.1 MB 27 kB/s 
     |████████████████████████████████| 14.7 MB 17.3 MB/s 
     |████████████████████████████████| 2.7 MB 41.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 

In [ ]:
!python -m transformers.onnx --model=./gpt-neo-therapist-small/ --feature=causal-lm-with-past --atol=5e-4 ./onnx/

In [ ]:
from optimum.onnxruntime.configuration import OptimizationConfig
from optimum.onnxruntime import ORTOptimizer

local_dir = "./gpt-neo-therapist-small/"

optimization_config = OptimizationConfig(optimization_level=99)

optimizer = ORTOptimizer.from_pretrained(
    local_dir,
    feature="causal-lm-with-past",
)
optimizer.export(
    onnx_model_path=f"{local_dir}model.onnx",
    onnx_optimized_model_output_path=f"{local_dir}model-optimized.onnx",
    optimization_config=optimization_config,
)

failed in shape inference <class 'AssertionError'>
failed in shape inference <class 'AssertionError'>
failed in shape inference <class 'AssertionError'>


Attention not fused


PosixPath('gpt-neo-therapist-small/model-optimized.onnx')

In [ ]:
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from optimum.onnxruntime import ORTQuantizer

In [ ]:
# The type of quantization to apply
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained(local_dir, feature="causal-lm-with-past")

quantizer.export(
    onnx_model_path=f"{local_dir}/model-cpu-opt.onnx",
    onnx_quantized_model_output_path=f"model-quantized.onnx",
    quantization_config=qconfig,
)

PosixPath('model-quantized.onnx')

In [ ]:
!cd gpt-neo-therapist-small && git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   model-optimized.onnx



In [ ]:
repo = Repository(local_dir="./gpt-neo-therapist-small/")

In [ ]:
repo.push_to_hub(commit_message="onnx optimized model")

Upload file model-optimized.onnx:   0%|          | 32.0k/673M [00:00<?, ?B/s]

To https://huggingface.co/arampacha/gpt-neo-therapist-small
   8481d92..29f3c56  main -> main



'https://huggingface.co/arampacha/gpt-neo-therapist-small/commit/29f3c56461d5e17428e7e665b8fb1f05dd4f1942'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoConfig
from onnxruntime import InferenceSession

In [ ]:
prompt_text = f"Answer like a therapist:\n Should I go to therapy?\nAnswer: "

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("arampacha/gpt-neo-therapist-small")#, use_auth_token=True)
session = InferenceSession(f"{local_dir}/model.onnx")

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [ ]:
config = AutoConfig.from_pretrained(model_id)

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
import numpy as np

def get_empty_past(inputs, config):
    input_ids = inputs["input_ids"]
    batch_size = input_ids.shape[0]
    past_shape = [batch_size, config.num_heads, 0, config.hidden_size // config.num_heads]
    past_keys = [f"past_key_values.{i//2}.{'key' if i%2==0 else 'value'}" for i in range(config.num_layers*2)]
    empty_past = {k: np.empty(past_shape, np.float32) for k in past_keys}
    return empty_past

In [ ]:
inputs = tokenizer(prompt_text, return_tensors="np")
inputs["attention_mask"] = inputs["attention_mask"].astype(np.float32)
empty_past = get_empty_past(inputs, config)
output_names = ["logits"] + [f"present.{i//2}.{'key' if i%2==0 else 'value'}" for i in range(config.num_layers*2)]
outputs = session.run(output_names=output_names, input_feed={**inputs, **empty_past})

In [ ]:
tokenizer.decode(outputs[0].argmax(-1).squeeze())

' is this lot, "\n I be to a?\nIf: Yes It'

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(local_dir)

In [ ]:
torch_inputs = tokenizer(prompt_text, return_tensors="pt")
logits = model(**torch_inputs)["logits"]

In [ ]:
tokenizer.decode(logits.argmax(-1).squeeze())

' is this lot, "\n I be to a?\nIf: Yes It'

In [ ]:
%%timeit
model(**torch_inputs)

10 loops, best of 5: 177 ms per loop


In [ ]:
%%timeit
session.run(output_names=output_names, input_feed={**inputs, **empty_past})

10 loops, best of 5: 125 ms per loop


In [ ]:
session = InferenceSession(f"{local_dir}/model-cpu-opt.onnx")
res_o = session.run(output_names=output_names, input_feed={**inputs, **empty_past})

In [ ]:
tokenizer.decode(res_o[0].argmax(-1).squeeze())

' is this lot, "\n I be to a?\nIf: Yes It'

In [ ]:
%%timeit
session.run(output_names=output_names, input_feed={**inputs, **empty_past})

10 loops, best of 5: 91.7 ms per loop


Issues with quantization - output doesn't match

In [ ]:
session = InferenceSession(f"model-quantized.onnx")
res_q = session.run(output_names=output_names, input_feed={**inputs, **empty_past})
tokenizer.decode(res_q[0].argmax(-1).squeeze())

'/ that few is she\n we think on the when\n\n: I I'

In [ ]:
%%timeit
session.run(output_names=output_names, input_feed={**inputs, **empty_past})

10 loops, best of 5: 51 ms per loop
